In [1]:
import warnings
warnings.filterwarnings('ignore')

<font size="3" color="Teal"><b>Model 0 </b></font>

* **Data group 01** : Includes data from 2007/01/01 to 2023/03/29 including Energy Demand in the Zone, actual meteorological variables and holidays of Mexico.

## 1.   Importing Preprocessed data

<summary>
    <font size="4" color="SteelBlue"><b>1.1 Importing libraries and functions</b></font>
</summary>

In [2]:
# Basic libraries
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

# Normalization libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import pickle as pk

In [3]:
# Importing PCA functions for futher processing
%run "./funciones/Y00pcafunction.ipynb"

<summary>
    <font size="4" color="SteelBlue"><b>1.2 Importing Dataframe from 'group01_allq2.csv'</b></font>
</summary>

In [4]:
#Save new datafrime with dated index
df_energy = pd.read_csv("./inputs/group01_allq2.csv", parse_dates=True)
df_energy['Date_time'] =  pd.to_datetime(df_energy['Date_time'],format='%Y/%m/%d %H:%M:%S')
#df_energy['Date'] =  pd.to_datetime(df_energy['Date'],format='%Y/%m/%d')
df_energy.set_index("Date_time", inplace=True)
df_energy=df_energy.asfreq('h')

 <font size="3" color="Teal"><b>Exogenous Calendar Features </b></font>

 The following columns are included related to Date and  Holidays of Mexico: 
 'Day', 'Hour', 'Month'. Day stands for weekday (0 Monday to 6 Sunday)
 'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
       'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
       'Christmas_NY', 'After_Christmas_NY', 'Date_timed'


<font size="3" color="Teal"><b>Endogenous Feature</b></font>

* **Energy Demand** (MW): Load energy demand in GCRNO (Gerencia de Control de Noroeste)  zone from hour $i$ to hour $i+1$ of the corresponding date, for $i=0,\dots 23$.


<font size="3" color="teal"><b>Exogenous Meteorological Features</b></font>

* **Tmax-Cab**, **Tmin-Cab**, **Tmax-Hmo**, **Tmin-Hmo**, **Tmax-Obr**, **Tmin-Obr**,**Tmax-Lmo**, **Tmin-Lmo**, **Tmax-Cul**, **Tmin-Cul** ($^\circ$C): Maximum and minimum temperature in Caborca, Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.

* **Prec_Hmo_Mm**, **Prec_Obr_Mm**, **Prec_Lmo_Mm**, **Prec_Cul_Mm**  (mm/h): Precipitation in Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.


In [5]:
# Adding Date-Timed as 'duplicate' column for flexibility
df_energy['Date_timed'] = df_energy.index
df_energy.head(2)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,...,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY,Date_timed
Date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,1,0,2007-01-01 00:00:00
2007-01-01 01:00:00,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,1,0,2007-01-01 01:00:00


In [6]:
df_energy.drop('Year', axis=1, inplace= True)
#df_e.drop('year', axis=1, inplace=True)
df_energy.tail(2)

,Energy_Demand,Day,Hour,Month,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,...,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY,Date_timed
Date_time,,,,,,,,,,,,,,,,,,,,,
2023-03-29 22:00:00,2584.34,2,22,3,30.2,32.0,32.1,31.5,33.0,12.4,...,0,0,0,0,0,0,0,0,0,2023-03-29 22:00:00
2023-03-29 23:00:00,2492.83,2,23,3,30.2,32.0,32.1,31.5,33.0,12.4,...,0,0,0,0,0,0,0,0,0,2023-03-29 23:00:00


In [7]:
df_energy.shape

(142368, 29)

## 2.   Splitting data into training, test and validation sets

**Separation into validation and testing**
Validation takes into account the last 4 Months of available Data.

**Testing:** Takes into account 12 Months previous to the Validation Data.

**Training:** All other available data after splitting validation and testing.



In [8]:
from datetime import datetime, timedelta

# Define the split dates for the validation and testing sets
validation_end = df_energy.index.max() - relativedelta(months=4)
testing_end = validation_end - relativedelta(months=12)
print(f'validation starts ',validation_end,'\ntest starts: ', testing_end)

validation starts  2022-11-29 23:00:00 
test starts:  2021-11-29 23:00:00


In [9]:
# Split the data into training, validation, and testing sets
training_data = df_energy.loc[df_energy.index <= testing_end]
testing_data = df_energy.loc[(df_energy.index > testing_end) & (df_energy.index <= validation_end)]
validation_data = df_energy.loc[df_energy.index > validation_end]

# Print the sizes of the resulting sets
print(f'Training data size: {len(training_data)}')
print(f'Testing data size: {len(testing_data)}')
print(f'Validation data size: {len(validation_data)}')

Training data size: 130728
Testing data size: 8760
Validation data size: 2880


In [10]:
# Print the shape of each data set.
print('Training set starts:', training_data.index.min(), 
      'Training set ends:', training_data.index.max() )
print( 'Testing set starts:', testing_data.index.min(), 
       'Testing set ends:', testing_data.index.max())
print('Validation set starts:', validation_data.index.min(), 
      'Validation set ends:', validation_data.index.max() )


Training set starts: 2007-01-01 00:00:00 Training set ends: 2021-11-29 23:00:00
Testing set starts: 2021-11-30 00:00:00 Testing set ends: 2022-11-29 23:00:00
Validation set starts: 2022-11-30 00:00:00 Validation set ends: 2023-03-29 23:00:00


In [11]:
df_energy.columns

Index(['Energy_Demand', 'Day', 'Hour', 'Month', 'Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul', 'Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm',
       'Prec_Cul_Mm', 'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
       'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
       'Christmas_NY', 'After_Christmas_NY', 'Date_timed'],
      dtype='object')

## 3.    Feature engineering

The  <font size="3" color="palevioletred"><b>random features</b></font> of the training dataframe are recoded (via a principal component procedure) on the features:


* **PC1_MAX, PC2_MAX, PC1_TMIN, PC2_TMIN, PCI_PREC, PC2_PREC** 


The features **PC2_TMAX**  and **PC2_TMIN** will be ignored later to avoid linear redundance. Then, these and the rest of the features are rescaled into the closed interval $[-1,1]$. The parameters of both procedures are exported and then applied in the test and validation set.

<summary>
    <font size="4" color="teal"><b>3.1 PCA procedure </b></font>
</summary>

In [12]:
# Setting new columns from training set
pca_exogenas_train, pca_clima,scaler_model_clima= exogen_pca(training_data, 'Date_timed')
pca_exogenas_train.to_csv('./outputs/PCA/pca_exogenas_train.csv')
# Applying training parameters to:
## Test set  
pca_exogenas_test = transform_data(testing_data,'Date_timed',pca_clima,scaler_model_clima)
pca_exogenas_test.to_csv('./outputs/PCA/pca_exogenas_test.csv')
## Validation test
pca_exogenas_val = transform_data(validation_data,'Date_timed',pca_clima,scaler_model_clima)
pca_exogenas_val.to_csv('./outputs/PCA/pca_exogenas_val.csv')

In [13]:
# exporting processing parameters for use outside the notebook
pk.dump(pca_clima, open("./outputs/PCA/pca_clima.pkl","wb"))
pk.dump(scaler_model_clima, open("./outputs/PCA/scaler_model_clima.pkl","wb"))

In [14]:
#Check if folder exists
import os

folder_path = './outputs/DataProcessing/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f'Folder created at {folder_path}')
else:
    print(f'Folder already exists at {folder_path}')
    

Folder already exists at ./outputs/DataProcessing/


In [15]:
pca_exogenas_test.columns

Index(['PC1_Weather', 'PC2_Weather', 'Date_timed', 'Energy_Demand', 'Day',
       'Hour', 'Month', 'Tmax-Cab', 'Tmax-Hmo', 'Tmax-Obr', 'Tmax-Lmo',
       'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr', 'Tmin-Lmo', 'Tmin-Cul',
       'Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm', 'Prec_Cul_Mm',
       'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week', 'May_1s', 'May_10t',
       'Sept_16', 'Nov_2nd', 'Before_Christmas_NY', 'Christmas_NY',
       'After_Christmas_NY'],
      dtype='object')

<summary>
    <font size="4" color="teal"><b>3.2. Rescaling all features into a closed interval $[-1,1]$ </b></font>
</summary>

In [16]:

Features = ['Energy_Demand', 'Day', 'Hour', 'Month','PC1_Weather', 'PC2_Weather',
            'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week', 'May_1s', 'May_10t', 
            'Sept_16', 'Nov_2nd', 'Before_Christmas_NY', 'Christmas_NY', 'After_Christmas_NY']

In [17]:
import os

folder_path = './outputs/DataProcessing/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f'Folder created at {folder_path}')
else:
    print(f'Folder already exists at {folder_path}')


Folder already exists at ./outputs/DataProcessing/


In [18]:
# Rescaling training set
train = pca_exogenas_train
scalers = {}  # Un diccionario con los scalers

for attr in Features:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    s_s = scaler.fit_transform(train[attr].values.reshape(-1,1))
    scalers[attr] = scaler
    train[attr] = s_s.ravel()
#train.info()

# Applying training parameters to:
# Validation set 
val = pca_exogenas_val
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(val[attr].values.reshape(-1,1))
    val[attr] = s_s.ravel()
# Test set
test = pca_exogenas_test
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(test[attr].values.reshape(-1,1))
    test[attr] = s_s.ravel()

In [19]:
# exporting processing parameters for use outside the notebook
pk.dump(scalers, open("./outputs/DataProcessing/scalers.pkl","wb"))

<summary>
    <font size="4" color="teal"><b>3.3 Training set view</b></font>
</summary>

In [20]:
train

,PC1_Weather,PC2_Weather,Energy_Demand,Day,Hour,Month,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
0,0.342080,-0.932098,-0.847748,-1.0,-1.000000,-1.000000,21.0,22.0,25.0,30.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
1,0.342080,-0.932098,-0.866667,-1.0,-0.913043,-1.000000,21.0,22.0,25.0,30.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
2,0.342080,-0.932098,-0.881532,-1.0,-0.826087,-1.000000,21.0,22.0,25.0,30.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
3,0.342080,-0.932098,-0.905856,-1.0,-0.739130,-1.000000,21.0,22.0,25.0,30.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
4,0.342080,-0.932098,-0.923874,-1.0,-0.652174,-1.000000,21.0,22.0,25.0,30.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130723,-0.065175,-0.940153,-0.181982,-1.0,0.652174,0.818182,29.5,31.5,30.0,29.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130724,-0.065175,-0.940153,-0.198198,-1.0,0.739130,0.818182,29.5,31.5,30.0,29.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130725,-0.065175,-0.940153,-0.210360,-1.0,0.826087,0.818182,29.5,31.5,30.0,29.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130726,-0.065175,-0.940153,-0.232432,-1.0,0.913043,0.818182,29.5,31.5,30.0,29.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [21]:
#train[['DEMANDA',  'MES', 'DIA', 'HORA', 'PC1_CLIMA', 'PC2_CLIMA','LUNES_FESTIVO','NAVIDAD_Y_NEW_YEAR' ]]
train[['Energy_Demand', 'Day', 'Hour', 'Month', 'PC1_Weather', 'PC2_Weather' , 'Monday_Holiday', 'After_Christmas_NY' ]]

,Energy_Demand,Day,Hour,Month,PC1_Weather,PC2_Weather,Monday_Holiday,After_Christmas_NY
0,-0.847748,-1.0,-1.000000,-1.000000,0.342080,-0.932098,-1.0,-1.0
1,-0.866667,-1.0,-0.913043,-1.000000,0.342080,-0.932098,-1.0,-1.0
2,-0.881532,-1.0,-0.826087,-1.000000,0.342080,-0.932098,-1.0,-1.0
3,-0.905856,-1.0,-0.739130,-1.000000,0.342080,-0.932098,-1.0,-1.0
4,-0.923874,-1.0,-0.652174,-1.000000,0.342080,-0.932098,-1.0,-1.0
...,...,...,...,...,...,...,...,...
130723,-0.181982,-1.0,0.652174,0.818182,-0.065175,-0.940153,-1.0,-1.0
130724,-0.198198,-1.0,0.739130,0.818182,-0.065175,-0.940153,-1.0,-1.0
130725,-0.210360,-1.0,0.826087,0.818182,-0.065175,-0.940153,-1.0,-1.0
130726,-0.232432,-1.0,0.913043,0.818182,-0.065175,-0.940153,-1.0,-1.0


## 4. Setting the tensors for the encoder

<summary>
    <font size="3" color="palevioletred"><b>Hyperparameters</b></font>
</summary>

* **past** $= (24 \times 7) + 12 = 180$:  Number of instances considered before the day to forecast 

* **skip** $= 12$: Number of instances to skip between the past and the future, this is because CENACE personnel require a 12-hour window to process the information and estimate the forecast

* **future** $= 24$: Number of instances to forecast 



From hyperparameters we get:

* $m$ = Number of pieces of instances of size $past+skip+future$ that fit on the training, validation and test set, respectively.


<summary>
    <font size="3" color="palevioletred"><b>Tensors dimension</b></font>
</summary>

Past tensors:

$$X_{past,n,m}$$


* $n = 18$: number of features

Future tensors:

$$Y_{future,1,m}$$

* $1$ means that we only consider the load energy demand in the corresponding instance

<img src="img/tensors.png" alt="drawing" width="400"/>


<summary>
    <font size="3" color="palevioletred"><b>Past tensors features</b></font>
</summary>

<summary>
    <font size="4" color="teal"><b>4.1 Setting  $X,Y$ tensors for training, validation and test sets</b></font>
</summary>

In [22]:
# Setting hyperparameters #cambiar parametros, n_futuro 36, n_salto 0
n_pasado = (24 * 7) + 12
n_futuro = 36 #Adjusting to 36 prev 24
n_salto = 0 #adjusting to 0 inst 12

In [23]:
def divide_series(series, indicadores, n_pasado, n_futuro, n_salto, es_train=True):
    """
    n_pasado: number of past observations for the encoder
    n_futuro: number of future observations
    n_salto: from where future observations start to count
    """
    X, y = list(), list() # Vamos a crear listas y al final hacemos ndarrays
    generador = range(len(series)) if es_train else range(0, len(series), n_futuro)

    for ini in generador:
        fin_anterior = ini + n_pasado
        fin_actual = fin_anterior + n_salto + n_futuro
        if fin_actual > len(series):
            break
        pasado = series[ini: fin_anterior, :]
        if len(indicadores) > 0: # replace next day value in "Holidays" features
            for i in range(indicadores.shape[1]):
                pasado = np.c_[pasado, indicadores[fin_anterior + n_salto + 1, i] * np.ones((n_pasado, 1))]
        futuro = series[fin_anterior + n_salto: fin_actual,[0]]
        X.append(pasado)
        y.append(futuro)
            
    return np.array(X), np.array(y)

In [24]:

features = ['Energy_Demand', 'Day', 'Hour', 'Month', 'PC1_Weather', 'PC2_Weather']
                    
nominal_features =  ['Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
       'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
       'Christmas_NY', 'After_Christmas_NY']


In [25]:
#Aqui se dividiria
n_attr = len(features) + len(nominal_features)
# Setting the tensors
X_train, y_train = divide_series(train[features].values,train[nominal_features].values, n_pasado, n_futuro, n_salto)
X_val, y_val = divide_series(val[features].values, val[nominal_features].values, n_pasado, n_futuro, n_salto)
X_test, y_test = divide_series(test[features].values, test[nominal_features].values, n_pasado, n_futuro, n_salto, es_train=False)

In [26]:
print(X_test)

[[[-0.30990991 -0.66666667 -1.         ... -1.         -1.
   -1.        ]
  [-0.3536036  -0.66666667 -0.91304348 ... -1.         -1.
   -1.        ]
  [-0.39009009 -0.66666667 -0.82608696 ... -1.         -1.
   -1.        ]
  ...
  [-0.37457658 -0.66666667 -0.2173913  ... -1.         -1.
   -1.        ]
  [-0.35614865 -0.66666667 -0.13043478 ... -1.         -1.
   -1.        ]
  [-0.34248198 -0.66666667 -0.04347826 ... -1.         -1.
   -1.        ]]

 [[-0.25720721 -0.33333333  0.04347826 ... -1.         -1.
   -1.        ]
  [-0.23693694 -0.33333333  0.13043478 ... -1.         -1.
   -1.        ]
  [-0.20945946 -0.33333333  0.2173913  ... -1.         -1.
   -1.        ]
  ...
  [-0.29743694 -0.33333333  0.82608696 ... -1.         -1.
   -1.        ]
  [-0.3246036  -0.33333333  0.91304348 ... -1.         -1.
   -1.        ]
  [-0.35847748 -0.33333333  1.         ... -1.         -1.
   -1.        ]]

 [[-0.30598198  0.33333333 -1.         ... -1.         -1.
   -1.        ]
  [-0.345

In [27]:
X_train.shape

(130513, 180, 16)

In [28]:
X_val.shape

(2665, 180, 16)

In [29]:
X_test.shape

(238, 180, 16)

<summary>
    <font size="4" color="teal"><b>4.2  Exporting tensors</b></font>
</summary>

In [30]:
np.save('./outputs/split/X_train.npy', X_train)
np.save('./outputs/split/X_val.npy', X_val)
np.save('./outputs/split/X_test.npy', X_test)
np.save('./outputs/split/y_train.npy', y_train)
np.save('./outputs/split/y_val.npy', y_val)
np.save('./outputs/split/y_test.npy', y_test)

In [31]:
from IPython import display
display.Image("https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png", embed = True)

URLError: <urlopen error [Errno 110] Connection timed out>

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>